In [1]:
from tensorflow import keras
import tensorflow as tf
import re

from transformers import BertTokenizer, TFBertForSequenceClassification

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from torch import nn
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
import torch
from torch.utils.data import DataLoader
from transformers import BertModel, AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import BertModel
import torch.optim as optim
import numpy as np



In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import glob
file_names = glob.glob('/content/drive/MyDrive/GA Research Project - Covid Tweet Classification/Twitter_Covid_GAProject/Annotated Data/*.csv')
print(file_names)

Mounted at /content/drive
['/content/drive/MyDrive/GA Research Project - Covid Tweet Classification/Twitter_Covid_GAProject/Annotated Data/CNN_tweets_unique.csv', '/content/drive/MyDrive/GA Research Project - Covid Tweet Classification/Twitter_Covid_GAProject/Annotated Data/CDCFLu_tweets_unique.csv', '/content/drive/MyDrive/GA Research Project - Covid Tweet Classification/Twitter_Covid_GAProject/Annotated Data/CDCtravel_tweets_unique_.csv', '/content/drive/MyDrive/GA Research Project - Covid Tweet Classification/Twitter_Covid_GAProject/Annotated Data/CDCemergency_tweets_unique.csv', '/content/drive/MyDrive/GA Research Project - Covid Tweet Classification/Twitter_Covid_GAProject/Annotated Data/CDCgov_tweets_unique.csv']


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_data = []
for file_name in file_names:
    temp = pd.read_csv(file_name)
    if 'CDCFLu' in file_name:
        temp = temp.head(50)
    else:
        temp = temp.head(499)
    df_data.append(temp)

In [5]:
df_data = pd.concat(df_data)

In [6]:
print(df_data.shape)

(2046, 13)


In [7]:
print(df_data["Category"].unique())

[nan 'Outbreak' 'Vaccine; Outbreak' 'Outbreak; Prevention; Vaccine'
 'Outbreak; Vaccine' 'Outbreak; Symptoms' 'Outbreak; Prevention' 'Vaccine'
 'Prevention' 'Outbreak; Vaccine; Prevention' 'Vaccine; Prevention'
 'Vaccine, Prevention; Symptoms' 'Vaccine, Outbreak' 'Travel; Outbreak'
 'Travel' 'Prevention, Travel' 'Travel; Prevention' 'Prevention; Travel'
 'Symptoms' 'Prevention; Outbreak' 'Prevention: Travel'
 'Prevention; travel' 'Prevntion' 'Outbreak ' 'Vaccine, Travel'
 'Outbreak; Travel' 'outbreak' 'Travel; prevention' 'Travel; Symptoms'
 'Ountbreak' 'outbreak ' 'prevention' ' Outbreak; Prevention'
 ' Prevention' 'Outbrek; Symptoms' 'Outbreak; symptoms'
 'Outbreak; prevention' 'prevention; outbreak' 'Prevention; Outbreak '
 'Outbreak Prevention' 'outbreak, prevention' 'Prevention; outbreak'
 'outbreak; prevention' 'travel; prevention' 'Symptoms; Prevention'
 'Prevention, Outbreak' 'Outbreak; Awarness' 'Prevention; Awarness'
 'Symptoms, Awarness' 'Outbreak;Awareness' 'Outbreak; reaso

In [8]:
def clean_text(text):
    text = re.sub(r'RT\s@\w+:\s', '', text) # remove retweet text
    text = re.sub(r'https?:\/\/\S+', '', text) # remove URLs
    text = re.sub(r'&\w+;', '', text) # remove HTML entities
    text = re.sub(r'\d+', '', text) # remove numbers
    text = re.sub(r'\W+', ' ', text) # remove special characters
    text = text.lower() # convert to lowercase
    return text

In [9]:
df_data["clean_text"] = df_data["text"].apply(clean_text)


# Filtering the rare terms.
df_data_filtered = df_data.groupby("Category").filter(lambda x: len(x) >1)
df_data_filtered.shape
print(df_data_filtered["Category"])
df_data_filtered["Category"]=[''.join(i.split()).lower() for i in df_data_filtered["Category"]]


2                  Outbreak
4                  Outbreak
12        Vaccine; Outbreak
28        Outbreak; Vaccine
39                 Outbreak
               ...         
487    Prevention; Awarness
488              Prevention
490                Symptoms
492              Prevention
496                Symptoms
Name: Category, Length: 938, dtype: object


In [10]:
def filter_category(categories):
  #options = ['travel', 'outbreak','prevention','vaccine','symptoms']
  if ',' in categories:
    category_list = categories.split(',')
  elif ';' in categories:
    category_list = categories.split(';')
  else:
    category_list = [categories]

  print(category_list)
  '''
  temp = []
  for item in category_list:
    item = item.strip().lower()
    if 'awarness' == item:
      continue
    temp.append(item.strip())
  #print('temp =', temp)
  '''
  return category_list

In [ ]:
df_data_filtered["Category"] = df_data_filtered["Category"].apply(filter_category)

labels=[item for item in df_data_filtered["Category"]]

#print(len(labels))

In [12]:
mlb = MultiLabelBinarizer()
one_hot_encodings = mlb.fit_transform(labels)

pd.DataFrame(one_hot_encodings, columns=mlb.classes_)
final_text=[text for text  in df_data_filtered["clean_text"]]

In [13]:
labels=[lab for lab  in one_hot_encodings]

lab,c =np.unique(one_hot_encodings, axis=0, return_counts=True)

print(lab,c)
print(" ")
print(mlb.classes_)
print(" ")
print("total",sum(c))
print(" ")
print(final_text[0])
print(df_data_filtered.columns)

[[0 0 0 0 0 1]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 1]
 [0 0 1 0 1 0]
 [0 0 1 1 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 1]
 [0 1 0 0 1 0]
 [0 1 0 1 0 0]
 [0 1 1 0 0 0]
 [1 0 1 0 0 0]
 [1 1 0 0 0 0]] [ 12 142  18 207   2  50   2 341  27   8  11 100   5  13]
 
['awarness' 'outbreak' 'prevention' 'symptoms' 'travel' 'vaccine']
 
total 938
 
b europe is now trading more goods with china than the united states a sign of how the pandemic is transforming the global economy the overall value of eu china goods trade last year was billion about billion more than between the eu and the us 
Index(['id', 'Account Created At(same for all rows)', 'status created_at',
       'Fav_cont', 'retweeted_count', 'reply_to_status', 'reply_to_screen',
       'quote', 'No', 'Category', 'text', 'Yes/No', 'Unnamed: 0',
       'clean_text'],
      dtype='object')


In [14]:
lnew=np.zeros((len(labels),5));
#print("before",labels)
for i,l in enumerate(labels):
  lnew[i,:]=l[1:]
final_label=lnew
print(final_label)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [15]:
outbreak=[]
prevention=[]
combined=[]
travel=[]
resources=[]
o_y=[]
p_y=[]
c_y=[]
t_y=[]
r_y=[]
for item,text in zip(final_label,final_text):
  if item[0]==1:
    outbreak.append(text)
    o_y.append([0,0])
  if item[1]==1:
    prevention.append(text)
    p_y.append([0,1])
  if item[2]==1 or item[4]==1:# consequences /resources
    combined.append(text)
    c_y.append([1,0])
  if item[3]==1:
    travel.append(text)
    t_y.append([1,1])
all_text=[]
all_text=outbreak+prevention+combined+travel

In [16]:
# Separate data by category
outbreak_text = [text for text, label in zip(df_data_filtered["clean_text"], final_label) if label[0] == 1]
prevention_text = [text for text, label in zip(df_data_filtered["clean_text"], final_label) if label[1] == 1]
combined_text = [text for text, label in zip(df_data_filtered["clean_text"], final_label) if label[2] == 1 or label[4] == 1]
travel_text = [text for text, label in zip(df_data_filtered["clean_text"], final_label) if label[3] == 1]


In [17]:
# Separate labels
outbreak_label = [[0, 0]] * len(outbreak_text)  # All labels for outbreak are [0, 0]
prevention_label = [[0, 1]] * len(prevention_text) # All labels for prevention are [0, 1]
combined_label = [[1,0]] * len(combined_text)
travel_label = [[1,1]] * len(travel_text)

In [18]:
## Convert labels to a NumPy array
o_y_array = np.array(outbreak_label)
p_y_array = np.array(prevention_label)
c_y_array = np.array(combined_label)
t_y_array = np.array(travel_label)

In [19]:
print(o_y_array.shape)
print(p_y_array.shape)
print(c_y_array.shape)
print(t_y_array.shape)

(500, 2)
(366, 2)
(72, 2)
(200, 2)


In [20]:
# Concatenate labels
all_labels = np.concatenate((o_y_array, p_y_array, c_y_array, t_y_array), axis=0)

# Concatenate text data
all_text_data = outbreak_text + prevention_text + combined_text + travel_text
len(all_labels),len(all_text_data)

(1138, 1138)

In [21]:
all_labels[:2], all_text_data[:2]

(array([[0, 0],
        [0, 0]]),
 ['b europe is now trading more goods with china than the united states a sign of how the pandemic is transforming the global economy the overall value of eu china goods trade last year was billion about billion more than between the eu and the us ',
  'b ryan reynolds and blake lively have once again opened their hearts and checkbooks to help those affected by the pandemic fueled hunger crisis the celebrity couple donated to two organizations feeding america and food banks canada '])

In [22]:
# Split data into training and testing sets
X_train_text, X_test_text, y_train, y_test = train_test_split(all_text_data, all_labels, test_size=0.2, random_state=95)

In [23]:
len(X_train_text), len(y_train), len(X_test_text), len(y_test)

(910, 910, 228, 228)

In [24]:
from transformers import BertTokenizer,BertModel
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [25]:
# Define a function to generate BERT embeddings for text data
def generate_bert_embeddings(text_data):
    # Tokenize text data
    tokenized_data = tokenizer(text_data, padding=True, truncation=True, return_tensors='pt')

    # Generate BERT embeddings
    with torch.no_grad():
        outputs = model(**tokenized_data)

    # Extract BERT embeddings from the output
    bert_embeddings = outputs.last_hidden_state.numpy()

    return bert_embeddings

In [26]:
type(X_train_text),type(y_train),type(X_test_text), type(y_test)

(list, numpy.ndarray, list, numpy.ndarray)

In [27]:
# Convert text data to BERT embeddings
X_train_embeddings = generate_bert_embeddings(X_train_text)
X_test_embeddings = generate_bert_embeddings(X_test_text)

In [ ]:
X_train_embeddings.shape,X_test_embeddings.shape,y_train.shape,y_test.shape

((910, 96, 768), (228, 55, 768), (910, 2), (228, 2))

In [29]:
from sklearn.utils.class_weight import compute_class_weight

In [30]:
# Calculate class weights for each class dimension
class_weights_binary = {}
for i in range(y_train.shape[1]):
  class_weights_binary[i] = compute_class_weight('balanced', classes=[0, 1], y=y_train[:, i])

In [ ]:
  # Convert class weights arrays to scalars
class_weights_binary = {class_idx: np.mean(weights) for class_idx, weights in class_weights_binary.items()}

In [ ]:
class_weights_binary

{0: 1.4226469031961022, 1: 1.0003914102365856}

In [ ]:
max_length = 0

# Iterate through all text samples
for text in all_text_data:
    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    # Update max_length if the length of tokens is greater than the current max_length
    max_length = max(max_length, len(tokens))

print("Maximum sequence length:", max_length)

Maximum sequence length: 94


In [ ]:
# # Convert BERT embeddings and labels into PyTorch tensors
# X_train_tensors = torch.tensor(X_train_embeddings, dtype=torch.float32)
# y_train_tensors = torch.tensor(y_train, dtype=torch.float32)
# X_test_tensors = torch.tensor(X_test_embeddings, dtype=torch.float32)
# y_test_tensors = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# # Define dataset and dataloader for training and testing
# train_dataset = TensorDataset(X_train_tensors, y_train_tensors)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# test_dataset = TensorDataset(X_test_tensors, y_test_tensors)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, text_data, labels, tokenizer, max_length):
        self.text_data = text_data
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        text = self.text_data[idx]
        label = self.labels[idx]

        # Tokenize text
        encoded_input = self.tokenizer(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = encoded_input['input_ids'].squeeze(0)  # Remove batch dimension
        attention_mask = encoded_input['attention_mask'].squeeze(0)  # Remove batch dimension

        label = torch.tensor(label, dtype=torch.float32)

        return input_ids, attention_mask, label


In [ ]:
# Define train and test datasets
train_dataset = CustomDataset(X_train_text, y_train, tokenizer, max_length)
test_dataset = CustomDataset(X_test_text, y_test, tokenizer, max_length)

In [ ]:
# Define batch size
batch_size = 32
# Create DataLoader objects for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class BERTClass(nn.Module):
    def __init__(self, num_classes=1):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids, attention_mask=attention_mask)
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output


In [ ]:
def train_model(model, train_loader, optimizer, criterion, device, class_weights=None):
    model.train()
    losses = []
    correct_predictions = 0
    total_samples = 0

    for batch in train_loader:
        input_ids, attention_mask, targets = batch  # Unpack the batch

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask)  # Pass attention_mask to the model
        loss = criterion(outputs, targets)

        if class_weights is not None:
          # Apply class weights if provided
          #weights = torch.tensor([class_weights[int(label)] for label in targets.tolist()], dtype=torch.float32).to(device)
          weights = torch.tensor([class_weights[label.item()] for label in targets], dtype=torch.float32).to(device)
          loss = (loss * weights).mean()

        loss.backward()
        optimizer.step()

        # Compute accuracy
        predictions = (outputs > 0.5).float()
        correct_predictions += (predictions == targets).sum().item()
        total_samples += targets.size(0)

        losses.append(loss.item())

    accuracy = correct_predictions / total_samples
    mean_loss = np.mean(losses)

    return mean_loss, accuracy


In [ ]:
def eval_model(model, val_loader, criterion, device):
    model.eval()
    losses = []
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids, targets = batch
            input_ids = input_ids.to(device)
            targets = targets.to(device)

            outputs = model(input_ids)
            loss = criterion(outputs, targets)

            losses.append(loss.item())

            predictions = (outputs > 0.5).float()
            all_predictions.extend(predictions.cpu().detach().numpy())
            all_targets.extend(targets.cpu().detach().numpy())

    avg_loss = np.mean(losses)

    # Convert predictions and targets to numpy arrays
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    # Calculate precision, recall, F1 score, and accuracy
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='binary')
    accuracy = accuracy_score(all_targets, all_predictions)

    # Calculate micro and macro average F1 scores
    f1_micro = f1_score(all_targets, all_predictions, average='micro')
    f1_macro = f1_score(all_targets, all_predictions, average='macro')

    return avg_loss, precision, recall, f1_micro, f1_macro, accuracy

In [ ]:
# Initialize model, optimizer, and loss function
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize BERT model
model = BERTClass(num_classes=2).to(device)  # Assuming binary classification

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-5)  # Adjust learning rate as needed

# Define loss function
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss with logits

In [ ]:
NUM_EPOCHS = 10  # Define the number of epochs

# Define lists to store training history
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []


In [ ]:
# Training loop
for epoch in range(NUM_EPOCHS):
    # Training
    train_loss, train_accuracy = train_model(model, train_loader, optimizer, criterion, device, class_weights=class_weights_binary)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    test_loss, test_accuracy = eval_model(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    # Print epoch-wise training statistics
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {test_loss:.4f}, Val Accuracy: {test_accuracy:.4f}')
